# **ONSAGER TESTING**

## **Installs**

### Firedrake

In [22]:
try:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh" -O "/tmp/firedrake-install.sh"
    !bash "/tmp/firedrake-install.sh"
    from firedrake import *  # noqa: F401
except:
    from firedrake import *  # noqa: F401

--2026-02-04 16:17:19--  https://fem-on-colab.github.io/releases/firedrake-install-development-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4775 (4.7K) [application/x-sh]
Saving to: ‘/tmp/firedrake-install.sh’

/tmp/firedrake-inst 100%[===================>]   4.66K  --.-KB/s    in 0s      

2026-02-04 16:17:19 (69.8 MB/s) - ‘/tmp/firedrake-install.sh’ saved [4775/4775]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FIREDRAKE_INSTALLED=/usr/local/share/fem-on-colab/firedrake.installed
+ [[ ! -f /usr/local/share/fem-on-colab/firedrake.installed ]]
+ set +x
























#########################################################

### Gmsh

In [21]:
try:
    !wget "https://fem-on-colab.github.io/releases/gmsh-install.sh" -O "/tmp/gmsh-install.sh"
    !bash "/tmp/gmsh-install.sh"
    import gmsh  # noqa: F401
except:
    import gmsh  # noqa: F401

--2026-02-04 16:17:13--  https://fem-on-colab.github.io/releases/gmsh-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3497 (3.4K) [application/x-sh]
Saving to: ‘/tmp/gmsh-install.sh’

/tmp/gmsh-install.s 100%[===================>]   3.42K  --.-KB/s    in 0s      

2026-02-04 16:17:13 (42.6 MB/s) - ‘/tmp/gmsh-install.sh’ saved [3497/3497]

+ INSTALL_PREFIX=/usr/local
++ awk -F/ '{print NF-1}'
++ echo /usr/local
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ GMSH_INSTALLED=/usr/local/share/fem-on-colab/gmsh.installed
+ [[ ! -f /usr/local/share/fem-on-colab/gmsh.installed ]]
+ H5PY_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/2ffb5740/releases/h5py-install.sh
+ [[ https://gith

### Other

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## **Code**

### Mesh

In [25]:
def rect_with_hole_mesh(r=0.2, left=1.0, right=2.0, updown=0.8, h=0.05):
    '''
    r = hole radius
    left = space to left of hole centre
    right = space to right of hole centre
    updown = space above / below hole centre
    h = (target) mesh size
    '''
    gmsh.initialize()
    gmsh.model.add("rect_with_hole")

    # Derived rectangle bounds
    xmin = -left
    xmax = right
    ymin = -updown
    ymax = updown

    # -------------------
    # Rectangle
    # -------------------
    p1 = gmsh.model.geo.addPoint(xmin, ymin, 0, h)
    p2 = gmsh.model.geo.addPoint(xmax, ymin, 0, h)
    p3 = gmsh.model.geo.addPoint(xmax, ymax, 0, h)
    p4 = gmsh.model.geo.addPoint(xmin, ymax, 0, h)

    l_bottom = gmsh.model.geo.addLine(p1, p2)
    l_right  = gmsh.model.geo.addLine(p2, p3)
    l_top    = gmsh.model.geo.addLine(p3, p4)
    l_left   = gmsh.model.geo.addLine(p4, p1)

    rect_loop = gmsh.model.geo.addCurveLoop(
        [l_bottom, l_right, l_top, l_left]
    )

    # -------------------
    # Circular hole (centred at origin)
    # -------------------
    c  = gmsh.model.geo.addPoint(0, 0, 0, h)
    cp = gmsh.model.geo.addPoint( r, 0, 0, h)
    cn = gmsh.model.geo.addPoint(-r, 0, 0, h)
    ct = gmsh.model.geo.addPoint(0,  r, 0, h)
    cb = gmsh.model.geo.addPoint(0, -r, 0, h)

    a1 = gmsh.model.geo.addCircleArc(cp, c, ct)
    a2 = gmsh.model.geo.addCircleArc(ct, c, cn)
    a3 = gmsh.model.geo.addCircleArc(cn, c, cb)
    a4 = gmsh.model.geo.addCircleArc(cb, c, cp)

    hole_loop = gmsh.model.geo.addCurveLoop([a1, a2, a3, a4])

    # -------------------
    # Surface with hole
    # -------------------
    surface = gmsh.model.geo.addPlaneSurface([rect_loop, hole_loop])

    gmsh.model.geo.synchronize()

    # -------------------
    # Physical groups (THIS is the important bit)
    # -------------------
    gmsh.model.addPhysicalGroup(1, [l_left], tag=1)
    gmsh.model.setPhysicalName(1, 1, "inlet")

    gmsh.model.addPhysicalGroup(1, [l_right], tag=2)
    gmsh.model.setPhysicalName(1, 2, "outlet")

    gmsh.model.addPhysicalGroup(1, [l_top, l_bottom], tag=3)
    gmsh.model.setPhysicalName(1, 3, "wall")

    gmsh.model.addPhysicalGroup(1, [a1, a2, a3, a4], tag=4)
    gmsh.model.setPhysicalName(1, 4, "hole")

    gmsh.model.addPhysicalGroup(2, [surface], tag=5)

    # -------------------
    # Mesh
    # -------------------
    gmsh.model.mesh.generate(2)
    gmsh.write("rect_with_hole.msh")
    gmsh.finalize()

    return Mesh("rect_with_hole.msh")

In [18]:
from netgen.geom2d import SplineGeometry
from firedrake import Mesh

L = 1.0
r = 0.2
h = 0.05

geo = SplineGeometry()

geo.AddRectangle(
    p1=(-L, -L),
    p2=( L,  L),
    bc="outer"
)

geo.AddCircle(
    c=(0.0, 0.0),
    r=r,
    leftdomain=0,
    rightdomain=1,
    bc="hole"
)

ngmesh = geo.GenerateMesh(maxh=h)
mesh = Mesh(ngmesh)


TypeError: expected str, bytes or os.PathLike object, not Mesh

In [ ]:
def navier_stokes(h=2**-10, degree=2, nu=0.01, timestep=2**-5, end_time=1.0):
    mesh = RectangleMesh(50, 50, 2.0, 1.0)
    cylinder = Circle(Point(0.5, 0.5), 0.1)
    mesh = MeshGenerator(mesh, [cylinder])
    n = FacetNormal(mesh)
    x, y = SpatialCoordinate(mesh)

    V = VectorFunctionSpace(mesh, 'CG', degree)
    Q = FunctionSpace(mesh, 'CG', degree - 1)
    W = V * Q

    u = Function(V)
    p = Function(Q)
    u_old = Function(V)

    bc_inlet = DirichletBC(W.sub(0), Constant((1.0, 0.0)), 'near(x[0], 0)')
    bc_walls = DirichletBC(W.sub(0), Constant((0.0, 0.0)), 'on_boundary')
    bcs = [bc_inlet, bc_walls]

    F = (inner((u - u_n) / timestep, v) * dx +
         inner(grad(u) * u, v) * dx +
         inner(grad(v), p) * dx -
         inner(div(u), q) * dx)

    sp = {
        'snes_max_it': 100,
    }

    fig, ax = plt.subplots(figsize=(10, 6))
    energy = []
    state = {'t': 0.0}

    def update(frame):
        if frame > 0:
            state['t'] += timestep
            print(f'Solving for time t = {state["t"]:.4f}:')
            solve(F == 0, u, bcs=bcs, solver_parameters=sp)
            u_n.assign(u)

            # Record energy
            energy.append(0.5 * assemble(inner(u, u) * dx))

        ax.clear()
        plot(u, axes=ax, linewidth=3)
        ax.set_title(f'Navier-Stokes (t = {state["t"]:.2f})')
        ax.set_xlabel('x')
        ax.set_ylabel('u')
        ax.set_ylim(-1, 1.5)
        ax.grid(True)

    num_frames = int(end_time / timestep) + 1
    anim = FuncAnimation(fig, update, frames=num_frames, interval=100)
    plt.close()

    # Plot energy
    plt.figure()
    plt.plot(np.arange(len(energy)) * timestep, energy)
    plt.title('Energy over time')
    plt.xlabel('Time')
    plt.ylabel('Energy')
    plt.grid(True)
    plt.show()

    return HTML(anim.to_jshtml())
